In [16]:
%matplotlib widget
from essential_func import *
from analysis_data import *
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])
os.chdir('/Users/bashit.a/Documents/Alzheimer/Dec-15/')     # location where your h5 and exp file are

In [33]:
%matplotlib widget

file = '2048_B8_masked.h5'
QSearchStart, QSearchEnd = 1.2, 1.3   # scaling regions 295,310 or may be 290,370
frame = 2080                           # amyloid frame    2428, 3101, 3923, 2190,  691, 1134, 458
tissue_frame = 60                     # tissue frame
bkg_frame = 2223                      # background frame
seek_mf = (-8,8,0.01)                 # mf goes from -8 to +8 by 0.01
window_size=4                         # window size for moving average filter

### Tissue scaled background subtraction
data = Data_Analysis(file, qgrid2)
IqBS = data.bkg_sub(bkg_frame = bkg_frame)
mf, area = data.scaling_frame(frame, tissue_frame, mf_Qindices = (QSearchStart, QSearchEnd), seek_mf = seek_mf, window_size=window_size, show_result=True)
data.plot()

MF =  2.1 AREA: 1.3~1.42 =  0.00037056708258545413 MSE =  1.7489100916011204e-06


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/bashit.a/Documents/Alzheimer/Codes/analysis_data.py:101: RuntimeWarning: divide by zero encountered in log
  axs[0,1].plot(self.qgrid, np.log(self.IqBS[self.input_fr]))


In [27]:
### Load all frames mf, and area data

sc_data = {}; sc_data['mf'] = np.zeros(data.n_patterns); sc_data['area'] = np.zeros(data.n_patterns)
for frame in range(data.n_patterns):
    sc_data['mf'][frame] , sc_data['area'][frame]   =  data.scaling_frame(frame, tissue_frame, mf_Qindices = (QSearchStart, QSearchEnd), seek_mf = seek_mf, window_size=window_size, show_result=False)

In [38]:
%matplotlib widget
### ------------- Draw heat map on descending area ------------- ###

sorted_area_idx  = np.argsort(sc_data['area'],kind='stable')[::-1][:50]

diff_patterns = np.zeros((data.n_patterns))
diff_patterns[sorted_area_idx] = sc_data['area'][sorted_area_idx]

Width, Height = width_height(file)
img_orig = snaking(Width, Height, diff_patterns)

f,ax = plt.subplots()
plot_heat_map_from_data(img_orig, Width, Height, args = (f, ax), title= f'Area Map {file}')
[print('Fr-',idx, 'Area: ' , np.round(diff_patterns[idx],8)) for idx in sorted_area_idx]
pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fr- 2080 Area:  0.00037057
Fr- 2079 Area:  0.00031173
Fr- 2067 Area:  0.00028504
Fr- 2081 Area:  0.00025967
Fr- 2068 Area:  0.00023579
Fr- 2066 Area:  0.00022853
Fr- 1958 Area:  0.00022327
Fr- 4620 Area:  0.00017242
Fr- 1957 Area:  0.00016782
Fr- 4621 Area:  0.00015426
Fr- 4529 Area:  0.00015317
Fr- 4528 Area:  0.00015036
Fr- 4622 Area:  0.00013588
Fr- 2078 Area:  0.00011829
Fr- 1959 Area:  0.00010327
Fr- 4530 Area:  8.16e-05
Fr- 458 Area:  7.994e-05
Fr- 4619 Area:  7.565e-05
Fr- 2189 Area:  7.473e-05
Fr- 1956 Area:  6.401e-05
Fr- 4527 Area:  6.154e-05
Fr- 4623 Area:  5.463e-05
Fr- 2190 Area:  5.427e-05
Fr- 2069 Area:  5.298e-05
Fr- 520 Area:  5.285e-05
Fr- 2429 Area:  5.019e-05
Fr- 3802 Area:  4.969e-05
Fr- 1476 Area:  4.944e-05
Fr- 4931 Area:  4.767e-05
Fr- 750 Area:  4.757e-05
Fr- 1475 Area:  4.721e-05
Fr- 177 Area:  4.669e-05
Fr- 4135 Area:  4.616e-05
Fr- 4808 Area:  4.538e-05
Fr- 2451 Area:  4.448e-05
Fr- 2428 Area:  4.448e-05
Fr- 1468 Area:  4.437e-05
Fr- 459 Area:  4.425e-05
Fr-

In [39]:
#### ------------ check heat maps data ------------ ####
%matplotlib widget
def exec_dropdown_value(frame):
    
    mf,area =  data.scaling_frame(frame, tissue_frame, mf_Qindices = (QSearchStart, QSearchEnd), seek_mf = seek_mf, window_size=window_size, show_result=True)
    data.plot()
    
frame = ipywidgets.Dropdown(options= sorted_area_idx,  description='Frame:',    disabled=False, continuous_update=False)
ipywidgets.interact(exec_dropdown_value, frame=frame)
print(sorted_area_idx)

interactive(children=(Dropdown(description='Frame:', options=(2080, 2079, 2067, 2081, 2068, 2066, 1958, 4620, …

[2080 2079 2067 2081 2068 2066 1958 4620 1957 4621 4529 4528 4622 2078
 1959 4530  458 4619 2189 1956 4527 4623 2190 2069  520 2429 3802 1476
 4931  750 1475  177 4135 4808 2451 2428 1468  459 2188 3641 1942 4089
  894 1945 1698 4404 1841 3101 4624 4526]


In [10]:
%matplotlib widget

from tqdm import tqdm

# specs
file = '2048_B8_masked.h5'
Nsplits  = 2                                        # number of lines to fit
StartIdx = 4                                        # including start index
LastIdx  = 52                                       # Excluding last index
frame    = 2223
max_frames = 40                                    # first 100 frame values
Rxc_str = ['Rxc-1', 'Rxc-2',]

# new part
data = Data_Analysis(file, qgrid2)
IqBS = data.bkg_sub(bkg_frame = 2223)

df_IqBS = pd.DataFrame(IqBS)    # convert to dataframe
list_max_frames = df_IqBS.sort_values(5,ascending=False)[:max_frames].index.to_list()   # 5-->th column values sorted in descending order and taking max 40 frames intensity to fit lines except are zero
print(list_max_frames)
Rxc = {}
n_patterns = len(IqBS)

for rxc_str in Rxc_str:
    Rxc[rxc_str] = np.zeros(n_patterns)     # except list_max_frames all values are zero

    for frame in tqdm(list_max_frames):
        df = optimize_best_lines(IqBS[frame], qgrid2, Nsplits, LastIdx, StartIdx,)   # get dataframe for the optimized one
        try:
            Rxc[rxc_str][frame] = df.iloc[df['rsq'].idxmax()][rxc_str]    
        except:
            print(f'Frame = {frame} {rxc_str} optimization failed - setting it to zero')
            Rxc[rxc_str][frame] = 0                 # set to zero in nan is found in try section
        
# plot heat maps
Width, Height = width_height(file)
f, axs = plt.subplots(1, len(Rxc_str), num=f'{file} Heat maps', figsize=(10,5))
for i, rxc_str in enumerate(Rxc_str):
    diff_patterns = Rxc[rxc_str]
    img_orig = snaking(Width, Height, diff_patterns)
    
    plot_heat_map_from_data(img_orig, Width, Height, args = (f, axs[i]), title= f'{file}- {rxc_str}')

  0%|          | 0/40 [00:00<?, ?it/s]/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:690: RuntimeWarning: divide by zero encountered in log
  IqQ = np.log(IqBS*qgrid)                            # background subtracted Iq
  8%|▊         | 3/40 [00:00<00:01, 21.28it/s]

(4941, 690)
[2067, 1958, 1326, 2080, 1358, 2428, 2068, 4374, 2197, 2199, 4373, 1357, 2427, 2079, 1957, 2452, 2198, 1327, 4912, 2200, 1325, 4223, 2558, 4913, 4723, 2066, 4848, 4911, 1359, 4847, 2437, 2081, 2323, 4222, 2322, 2196, 1959, 2320, 2438, 2453]


/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:690: RuntimeWarning: invalid value encountered in log
  IqQ = np.log(IqBS*qgrid)                            # background subtracted Iq
 15%|█▌        | 6/40 [00:00<00:01, 21.39it/s]/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:717: RuntimeWarning: invalid value encountered in sqrt
  df.loc[idx,f'Rxc-{i+1}'] = np.round(np.sqrt(-df.loc[idx,f'slope-{i+1}']*2),4)
100%|██████████| 40/40 [00:01<00:00, 21.95it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
%matplotlib widget
# specs
frame = 2067
# verification
df = optimize_best_lines(IqBS[frame], qgrid2, Nsplits, LastIdx, StartIdx, print_summary=True, show_plot=True)
print(df)

Total Combinations = 45

Summary of results Rsq -- 
 0               4.0000
1              18.0000
2              18.0000
3              52.0000
Tpoints-1      14.0000
qgridL-1        0.0090
qgridH-1        0.0230
slope-1     -4833.9374
Rxc-1          98.3254
I(0)-1          0.9474
rsq-1           0.9472
std_err-1     329.6247
Tpoints-2      34.0000
qgridL-2        0.0230
qgridH-2        0.0640
slope-2      -698.0548
Rxc-2          37.3646
I(0)-2         -1.2604
rsq-2           0.9421
std_err-2      30.5975
rsq             1.8893
Name: 12, dtype: float64
More Summary of results Rsq -- 
     0   1   2   3  Tpoints-1  qgridL-1  qgridH-1     slope-1     Rxc-1  \
0   4   6   6  52          2     0.009     0.011 -12982.1460  161.1344   
1   4   7   7  52          3     0.009     0.012 -11530.7305  151.8600   
2   4   8   8  52          4     0.009     0.013 -10360.3461  143.9468   
3   4   9   9  52          5     0.009     0.014  -9429.4692  137.3279   
4   4  10  10  52          6     0.0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

    0   1   2   3  Tpoints-1  qgridL-1  qgridH-1     slope-1     Rxc-1  \
0   4   6   6  52          2     0.009     0.011 -12982.1460  161.1344   
1   4   7   7  52          3     0.009     0.012 -11530.7305  151.8600   
2   4   8   8  52          4     0.009     0.013 -10360.3461  143.9468   
3   4   9   9  52          5     0.009     0.014  -9429.4692  137.3279   
4   4  10  10  52          6     0.009     0.015  -8636.6298  131.4278   
5   4  11  11  52          7     0.009     0.016  -7878.8447  125.5296   
6   4  12  12  52          8     0.009     0.017  -7300.9062  120.8380   
7   4  13  13  52          9     0.009     0.018  -6807.7529  116.6855   
8   4  14  14  52         10     0.009     0.019  -6297.4656  112.2271   
9   4  15  15  52         11     0.009     0.020  -5884.7404  108.4872   
10  4  16  16  52         12     0.009     0.021  -5497.3718  104.8558   
11  4  17  17  52         13     0.009     0.022  -5129.8891  101.2906   
12  4  18  18  52         14     0.009

In [12]:
pd.set_option('display.max_columns',None)
df.style.background_gradient()
df[df['rsq'] > 1.86]                      # Nsplits
#df[df['rsq-3'] > 0.99]['Tpoints-3'].idxmax()      # Nsplits
#plot_linear_lines(xData, yData, best_indices)

,0,1,2,3,Tpoints-1,qgridL-1,qgridH-1,slope-1,Rxc-1,I(0)-1,rsq-1,std_err-1,Tpoints-2,qgridL-2,qgridH-2,slope-2,Rxc-2,I(0)-2,rsq-2,std_err-2,rsq
2,4,8,8,52,4,0.009,0.013,-10360.3461,143.9468,1.6805,0.9908,707.4576,44,0.013,0.064,-921.5366,42.9310,-0.7508,0.8726,54.3384,1.8634
3,4,9,9,52,5,0.009,0.014,-9429.4692,137.3279,1.5856,0.9869,626.9592,43,0.014,0.064,-893.2780,42.2677,-0.8131,0.8823,50.9652,1.8692
4,4,10,10,52,6,0.009,0.015,-8636.6298,131.4278,1.4986,0.9828,570.7240,42,0.015,0.064,-866.6187,41.6322,-0.8723,0.8912,47.8814,1.8740
5,4,11,11,52,7,0.009,0.016,-7878.8447,125.5296,1.4093,0.9761,551.1112,41,0.016,0.064,-840.9667,41.0114,-0.9297,0.9000,44.8855,1.8761
6,4,12,12,52,8,0.009,0.017,-7300.9062,120.8380,1.3361,0.9733,493.7115,40,0.017,0.064,-817.0807,40.4248,-0.9836,0.9078,42.2534,1.8811
7,4,13,13,52,9,0.009,0.018,-6807.7529,116.6855,1.2693,0.9709,445.6174,39,0.018,0.064,-794.8459,39.8709,-1.0342,0.9144,39.9717,1.8853
8,4,14,14,52,10,0.009,0.019,-6297.4656,112.2271,1.1953,0.9642,429.2031,38,0.019,0.064,-772.9985,39.3192,-1.0843,0.9214,37.6173,1.8856
9,4,15,15,52,11,0.009,0.020,-5884.7404,108.4872,1.1313,0.9607,396.8797,37,0.020,0.064,-752.8977,38.8046,-1.1309,0.9272,35.6586,1.8879
10,4,16,16,52,12,0.009,0.021,-5497.3718,104.8558,1.0673,0.9559,373.3956,36,0.021,0.064,-733.6226,38.3046,-1.1760,0.9327,33.8078,1.8886
11,4,17,17,52,13,0.009,0.022,-5129.8891,101.2906,1.0027,0.9497,355.9882,35,0.022,0.064,-714.7376,37.8084,-1.2206,0.9383,31.9132,1.8880


In [29]:
%matplotlib widget

# specs
frame = 617    # 2048_B8_masked frame = 617 Rxc_2 = nan
QQ  = np.square(qgrid2)                            # squaring q values
IqQ = np.log(IqBS[frame]*qgrid2)                         # background subtracted Iq

# semi specs
xData = QQ
yData = IqQ

# computation
def update_plot(Q1, Q2H):
    Q1L, Q1H= Q1
    plot_linear_lines(xData=xData, yData=yData, indices=[Q1L,Q1H,Q1H,Q2H])

Q1 =  ipywidgets.IntRangeSlider(min=0, max=109, value = [2, 50], step=1)
Q2H = ipywidgets.IntSlider(min=0, max=109, value = 109, step=1)

ipywidgets.interact(update_plot, Q1=Q1, Q2H=Q2H)

<ipython-input-29-1aa1b51b47a5>:6: RuntimeWarning: divide by zero encountered in log
  IqQ = np.log(IqBS[frame]*qgrid2)                         # background subtracted Iq
<ipython-input-29-1aa1b51b47a5>:6: RuntimeWarning: invalid value encountered in log
  IqQ = np.log(IqBS[frame]*qgrid2)                         # background subtracted Iq


interactive(children=(IntRangeSlider(value=(2, 50), description='Q1', max=109), IntSlider(value=109, descripti…

<function __main__.update_plot(Q1, Q2H)>

In [101]:
# 1-d plot for high intensities
%matplotlib widget
import mplcursors
from matplotlib.widgets import CheckButtons
frames = np.array(sorted(list_max_frames))
print(sorted(list_max_frames))

f, ax = plt.subplots(2,2, figsize = (16,8), num=f'data', constrained_layout = False) 
plt.subplots_adjust(left=0.15)

# [4912,]
# [4373, 4374,]
# [2427, 2428, 2452,]
# [2197, 2198, 2199, 2200,]
# [1957, 1958, 2067, 2068, 2079, 2080,]
# [1326, 1327, 1357, 1358, ]
# [1326, 1327, 1357, 1358, 1957, 1958, 2067, 2068, 2079, 2080, 2197, 2198, 2199, 2200, 2427, 2428, 2452, 4373, 4374, 4912]

saxs_plots = []    
waxs_plots = []
for frame in frames:  
    QQ  = np.square(qgrid2)                            # squaring q values
    IqQ = (IqBS[frame]*qgrid2)                         # background subtracted Iq

    # semi specs
    xData = QQ
    yData = IqQ
    
    saxs_plots.append(ax[0,0].plot(xData[130:], yData[130:], visible=False, label = f'{frame}' ))   # here dataset has only one frame
    ax[0,0].set_xlabel("$q (\AA^{-1})$",); 
    ax[0,0].set_ylabel("$I$",);
    ax[0,0].set_xscale('linear')
    ax[0,0].set_yscale('log')

    ax[1,0].set_xlabel("$q (\AA^{-1})$",); 
    ax[1,0].set_ylabel("$I$",);
    ax[1,0].set_xscale('linear')
    ax[1,0].set_yscale('log')
    waxs_plots.append(ax[1,0].plot(xData[2:50] , yData[2:50], visible=False,label=f'{frame}')); 
    
    plot_heat_map_from_data(img_orig, Width, Height, args = (f, ax[0,1]), )

#plt.tight_layout()

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.005, 0.1, 0.05, 0.6])
labels = [str(line[0].get_label()) for line in saxs_plots]   #label is a string
visibility = [line[0].get_visible() for line in saxs_plots]
check = CheckButtons(rax, labels, visibility, )
for r,l in zip(check.rectangles, check.lines):
    r.set_width(0.1)
    r.set_alpha(0.4)
    for ll in l:
        ll.set_linewidth(8)
    
# [ll.set_linewidth(8) for l in check.lines for ll in l]
def func(label):
    index = labels.index(label)
    saxs_plots[index][0].set_visible(not saxs_plots[index][0].get_visible())
    waxs_plots[index][0].set_visible(not waxs_plots[index][0].get_visible())
    
    ax[0,0].legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize='xx-small')
    ax[1,0].legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize='xx-small')

    plt.draw()

check.on_clicked(func)
plt.subplot_tool()
#mplcursors.cursor()

[1325, 1326, 1327, 1357, 1358, 1359, 1957, 1958, 1959, 2066, 2067, 2068, 2079, 2080, 2081, 2196, 2197, 2198, 2199, 2200, 2320, 2322, 2323, 2427, 2428, 2437, 2438, 2452, 2453, 2558, 4222, 4223, 4373, 4374, 4723, 4847, 4848, 4911, 4912, 4913]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
import mplcursors
from matplotlib.widgets import CheckButtons

class Check_Box_Plots():
    
    def __init__(self, X, Y, labels, ax, ): 
        
        # initialize variable
        self.plots = []
        self.X = X
        self.Y = Y
        self.labels = labels
        self.ax = ax
    
    def plot_data(self, xlabel="$q(\AA^{-1})$", ylabel="$I$", xscale='linear', yscale='linear'):
        # plots checkbox data without visibility
        for idx in range(len(self.Y)):
            self.plots.append(self.ax.plot(self.X, self.Y[idx], visible=True, label = f'{self.labels[idx]}' ))   # here dataset has only one frame
            self.ax.set(xlabel = xlabel, ylabel=ylabel, xscale=xscale, yscale=yscale)

    # call when checkbox items clicked
    def func(self,label):
        """
            only global variable is plots, labels, ax
        """
        index = self.labels.index(label)

        self.plots[index][0].set_visible(not self.plots[index][0].get_visible())                # visible plot when clicked
        self.ax.legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize='xx-small')      # visible lengend when clicked
        #plt.draw()

# instantiate object
# inputs data
X, Y =  np.square(qgrid2)    , IqBS[[2080, 1956]]*qgrid2 
labels = ['2080', '1956']
f,ax = plt.subplots(2,2,constrained_layout = False) 
plots = Check_Box_Plots(X, Y, labels, ax[1,1])
plots.plot_data()

# Make checkbuttons and attributes with all plotted lines with correct visibility
rax        = plt.axes([0.005, 0.1, 0.05, 0.6])              # checkbox axis location
labels     = [str(line[0].get_label()) for line in plots.plots]   # label is a string
visibility = [line[0].get_visible()    for line in plots.plots]   # visibility
check      = CheckButtons(rax, labels, visibility, )
for r,l in zip(check.rectangles, check.lines):
    r.set_width(0.1)                # checkbox width
    r.set_alpha(0.4)                
    for ll in l:
        ll.set_linewidth(8)         # check/uncheck linewidth or [ll.set_linewidth(8) for l in check.lines for ll in l]    

check.on_clicked(plots.func)              # check.get_status()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0

In [119]:
## Do clustering
import cv2
from scipy.ndimage import label

img_gray = np.asarray(img_orig, dtype=np.uint8)
_, thresh_img = cv2.threshold(img_gray, 0, 255,cv2.THRESH_BINARY); #plt.imshow(thresh_img, cmap='gray')
labeled_array, num_features = label(thresh_img, np.ones((3,3)))
print('Total clusters = ', num_features)

Width, Height = labeled_array.shape[1], labeled_array.shape[0]
sna = snaking(Width, Height , np.arange(0, Width*Height))

[print('label = ', i, sna[labeled_array==i]) for i in np.unique(labeled_array) if i!=0]

Total clusters =  11
label =  1 [4911 4912 4913 4848 4847]
label =  2 [4723]
label =  3 [4374 4373]
label =  4 [4223 4222]
label =  5 [2558]
label =  6 [2452 2453 2428 2427]
label =  7 [2438 2437 2320]
label =  8 [2322 2323]
label =  9 [2196 2197 2198 2199 2200]
label =  10 [2079 2080 2081 2068 2067 2066 1957 1958 1959]
label =  11 [1357 1358 1359 1327 1326 1325]


[None, None, None, None, None, None, None, None, None, None, None]

In [27]:
from generate_files import *
save_mat('IqBS', ('IqBS', 'qgrid2'), (IqBS, qgrid2))